In [ ]:
!pip install langchain boto3
!pip install transformers
!pip install sentence_transformers

In [ ]:
pip install langchain==0.0.224

In [ ]:
pip install boto3 --upgrade

In [ ]:
!pip install ~/SageMaker/botocore-1.29.162-py3-none-any.whl
!pip install ~/SageMaker/boto3-1.26.162-py3-none-any.whl

In [ ]:
pip install wikipedia

In [1]:
import boto3
import langchain
from langchain.retrievers import AmazonKendraRetriever
from typing import List
from typing import Dict
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
#from langchain import SagemakerEndpoint, LLMChain
from langchain import LLMChain
from langchain.llms.bedrock import Bedrock
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain
import json

from langchain.docstore.document import Document
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

In [2]:
langchain.debug = True

In [3]:
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"inputs": inputs, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
#        return response_json["vectors"]
        return response_json[0]["generated_text"]



content_handler = ContentHandler()


In [4]:
from langchain import PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

#llm2 = Bedrock(
#model_id="anthropic.claude-v1",
#model_kwargs={'max_tokens_to_sample': 300})

maxTokensToSample = 300
temp = 0
topP = 0.5
topK= 5

modelArgs = {'max_tokens_to_sample': int(maxTokensToSample), 'temperature':float(temp), "top_k":int(topK),"top_p": float(topP),"stop_sequences":["Question"]}
#modelArgs = {'maxTokens': int(maxTokensToSample), 'temperature':float(temp), "topP": float(topP),"stop_sequences":[]}
#modelArgs = {'maxTokens': int(maxTokensToSample) , 'temperature':float(temp), "topP": float(topP)}

llm2 = Bedrock(
model_id="anthropic.claude-instant-v1",
model_kwargs=modelArgs)

#llm2.debug = True

In [5]:
from langchain import Wikipedia
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer


docstore = DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search",
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup",
    ),
]



In [30]:
react = initialize_agent(tools, llm2, agent=AgentType.REACT_DOCSTORE, verbose=True)
#question = "what is the name of the first person to summit the worlds highest mountain?"
#question = "what is the name of the captain that won the 2019 cricket world cup ?"
question = "what is the name of the captain that won the 1996 cricket world cup ?"

# output prompt's template
#print(react.agent.llm_chain.prompt.template)



In [31]:
react.run(question)

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "what is the name of the captain that won the 1996 cricket world cup ?"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "what is the name of the captain that won the 1996 cricket world cup ?",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:Bedrock] Entering LLM run with input:
{
  "prompts": [
    "Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?\nThought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.\nAction: Search[Colorado orogeny]\nObservation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.\nThought: It does not mention the eastern sector. So 

'Arjuna Ranatunga'